# Generic Chains Overview

## Simple Chain

The most elementary type of chain is known as a basic chain, which represents the simplest form of crafting a chain. <br>In this setup, there is only one Language Model (LLM) responsible for receiving an input prompt and using it for generating text.

In [1]:
import os
import pprint as pp
from pathlib import Path
from dotenv import find_dotenv, load_dotenv

load_dotenv(Path('../../.env'))

from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [2]:
llm = OpenAI()

In [3]:
prompt = PromptTemplate(
    input_variables=["place"],
    template="Best places to visit in {place}?",
)

In [26]:
chain = LLMChain(llm=llm, prompt=prompt)

# Run the chain only specifying the input variable.
print(chain.run("India"))



1. Taj Mahal, Agra
2. Golden Temple, Amritsar
3. Goa
4. Jaipur
5. Udaipur
6. Varanasi
7. Ranthambore National Park
8. Kerala
9. Darjeeling
10. Leh-Ladakh


## Simple Sequential Chains

Sequential Chains involves making a series of consecutive calls to the language model.<br> This approach proves especially valuable when there is a need to utilize the output generated from one call as the input for another call.

In [21]:
from langchain.chains import SimpleSequentialChain
from langchain import HuggingFaceHub

In [27]:
template = """You have to suggest 5 best places to visit in {place}?

YOUR RESPONSE:
"""
prompt_template = PromptTemplate(
    input_variables=["place"], 
    template=template)

####

template = """Given a list a places, please estimate the expenses to visit all of them in local currency and also the days needed
{expenses}

YOUR RESPONSE:
"""
prompt_template = PromptTemplate(
    input_variables=["expenses"],
    template=template)

In [28]:
# Use a huggingface LLM for this chain
HF_llm = HuggingFaceHub(repo_id = "google/flan-t5-large")
# llm = OpenAI()

In [29]:
# Initializa chains
place_chain = LLMChain(llm=llm, prompt=prompt_template)
expenses_chain = LLMChain(llm=llm, prompt=prompt_template)

In [30]:
# Add them into a sequential chain
final_chain = SimpleSequentialChain(chains=[chain, place_chain, expenses_chain], verbose=True)
review = final_chain.run("India")



> Entering new SimpleSequentialChain chain...


1. Taj Mahal, Agra
2. Jaipur
3. Andaman and Nicobar Islands
4. Goa
5. Kerala
6. Leh and Ladakh
7. Rishikesh
8. Varanasi
9. Jim Corbett National Park
10. Darjeeling

The estimated expenses to visit all of these places in Indian Rupees would be around Rs. 1,25,000-1,50,000, depending on the mode of transport and accommodation you choose. The time needed to visit all these places would be around 10-12 days.
The estimated expenses to visit all of these places in Indian Rupees would be around Rs. 1,25,000-1,50,000, depending on the mode of transport and accommodation you choose. The time needed to visit all these places would be around 10-12 days.

> Finished chain.


In [20]:
pp.pprint(review)

('The estimated expenses to visit all of the places in India would depend on '
 'the places you intend to visit and the duration of the trip. On average, the '
 'cost of a trip to India can range from $50-$100 per day in local currency '
 '(Indian Rupees). The total number of days needed for the trip would also '
 'depend on how many places you plan to visit and how much time you plan to '
 'spend in each place.')
